- 식품의약품안전처 공공데이터 사이트에서 조리식품DB를 API로 받아올수 있음
-https://www.foodsafetykorea.go.kr/api/openApiInfo.do?menu_grp=MENU_GRP31&menu_no=661&show_cnt=10&start_idx=1&svc_no=COOKRCP01

- github에 식약청에서 API로 받아온 레시피 데이터를 활용
-https://github.com/tunEmvegnomb/cook_alone

In [1]:
# 데이터 가져오기
import pandas as pd
df = pd.read_csv("0_2000_parsed.csv", encoding="utf8", na_values="NA")

In [2]:
df.head(3)

,Unnamed: 0,RCP_SEQ,RCP_NM,RCP_WAY2,RCP_PAT2,INFO_WGT,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,...,MANUAL03,MANUAL_IMG03,MANUAL04,MANUAL_IMG04,MANUAL05,MANUAL_IMG05,MANUAL06,MANUAL_IMG06,RAW,parsed
0,0,18,버섯 두유 소스 볶음,볶기,반찬,NaN,285.0,17.0,9.0,20.0,...,3. 볶은 새송이버섯을 소스에 넣고 고루 섞어 가며 조금 더 볶아 접시에 담은 후 ...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,NaN,NaN,NaN,NaN,NaN,NaN,"['새송이버섯 100g(3개)', '올리브유 10g(2작은술)', '두유 20g(1...","['새송이버섯', '두유', '생크림', '청양고추', '새싹채소']"
1,1,19,백일송이버섯 볶음,볶기,반찬,NaN,80.0,10.0,3.0,3.0,...,3. 버섯이 살짝 숨이 죽으면 파프리카를 넣어 함께 볶아 접시에 담는다.c,http://www.foodsafetykorea.go.kr/uploadimg/coo...,NaN,NaN,NaN,NaN,NaN,NaN,"['백일송이버섯 65g(1컵)', '노랑 파프리카 5g(5×1cm)', '빨강 파프...","['백일송이버섯', '노랑 파프리카', '빨강 파프리카', '카놀라유']"
2,2,21,땅콩소스 버섯가지나물,끓이기,반찬,55.0,64.2,3.4,4.3,3.7,...,3. 가지와 버섯은 끓는 물에 각각 데친 후 찬물에 식힌다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,4. 땅콩소스 재료를 섞어 땅콩소스를 만든다.,http://www.foodsafetykorea.go.kr/uploadimg/202...,"5. 데친 가지와 버섯은 물기를 제거해 그릇에 담고, 땅콩소스를 곁들여 마무리한다.",http://www.foodsafetykorea.go.kr/uploadimg/202...,NaN,NaN,"['가지(15g)', '새송이버섯(15g)', '표고버섯(10g)', '땅콩가루(5...","['가지', '새송이버섯', '표고버섯', '땅콩가루', '다진마늘', '참기름',..."


In [3]:
# 필요한 열만 추출
df = df[['RCP_SEQ', 'RCP_NM', 'RCP_PAT2', 'ATT_FILE_NO_MAIN', 'RCP_PARTS_DTLS', 'MANUAL01', 'MANUAL02', 'MANUAL03','MANUAL04', 'MANUAL05', 'MANUAL06']]

In [4]:
# 조리순서 데이터 정제
df['cookstep'] = df['MANUAL01'].astype(str) + "\n" + df['MANUAL02'].astype(str) + "\n" + df['MANUAL03'].astype(str) + "\n" + df['MANUAL04'].astype(str) + "\n" + df['MANUAL05'].astype(str) + "\n" + df['MANUAL06'].astype(str)
df['cookstep'] = df['cookstep'].str.replace("\nnan", "")

In [5]:
# 필요한 데이터 생성
df['category'] = "일반식"
df['likes'] = 0
df['hits'] = 0

df['level'] = "초급"
df.loc[df['MANUAL05'].isna()==False, 'level'] = "중급"
df.loc[df['MANUAL06'].isna()==False, 'level'] = "고급"

In [6]:
# DB에 저장할 칼럼명으로 변경
df = df[['RCP_NM', 'category', 'ATT_FILE_NO_MAIN', 'level', 'RCP_PARTS_DTLS', 'cookstep', 'likes', 'hits']]
df.columns = ['brecipe_name', 'category', 'img_url', 'level', 'ingredient', 'cookstep', 'likes', 'hits']

In [7]:
df

,brecipe_name,category,img_url,level,ingredient,cookstep,likes,hits
0,버섯 두유 소스 볶음,일반식,http://www.foodsafetykorea.go.kr/uploadimg/coo...,초급,"버섯볶음\n새송이버섯 100g(3개), 올리브유 10g(2작은술)\n두유 소스\n두...",1. 새송이버섯은 굵게 채썬 후 달군 프라이팬에 기름을 두르고 노릇하게 볶는다.a\...,0,0
1,백일송이버섯 볶음,일반식,http://www.foodsafetykorea.go.kr/uploadimg/coo...,초급,"백일송이버섯볶음\n백일송이버섯 65g(1컵), 노랑 파프리카 5g(5×1cm), 빨...",1. 파프리카는 씨를 빼고 버섯 크기와 비슷하게 썬다.a\n2. 달구어진 팬에 카놀...,0,0
2,땅콩소스 버섯가지나물,일반식,http://www.foodsafetykorea.go.kr/uploadimg/202...,중급,"•필수 재료 : 가지(15g), 새송이버섯(15g), 표고버섯(10g)\n•땅콩 소...","1. 가지는 꼭지를 떼어 편으로 썬다.\n2. 새송이버섯은 길게 편으로 썰고, 표고...",0,0
3,새우 두부 계란찜,일반식,http://www.foodsafetykorea.go.kr/uploadimg/coo...,초급,"새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g...","1. 손질된 새우를 끓는 물에 데쳐 건진다.a\n2. 연두부, 달걀, 생크림, 설탕...",0,0
4,부추 콩가루 찜,일반식,http://www.foodsafetykorea.go.kr/uploadimg/coo...,초급,"[1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\n·양념장 : 저염간장 3g(...","1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣...",0,0
...,...,...,...,...,...,...,...,...
1105,새우채소 김치롤,일반식,https://www.foodsafetykorea.go.kr/common/ecmFi...,고급,"[ 2인분 ] 김치(¼포기), 오이(1개), 파프리카(노랑, 빨강 각 1개), 깻잎...","1. 김치는 깨끗하게 씻어 찬물에 30분간 담근 뒤 건져내고, 물기를 꼭 짠 뒤 윗...",0,0
1106,생선까스&타르타르소스,일반식,http://www.foodsafetykorea.go.kr/uploadimg/202...,고급,"•필수재료 : 명태포(40g), 튀김가루(20g), 달걀(1/4개), 식용유(5g)...",1. 명태포는 키친타월에 올려 물기를 제거한다.\n2. 후춧가루와 레몬즙을 뿌려 비...,0,0
1107,느타리버섯볶음,일반식,http://www.foodsafetykorea.go.kr/uploadimg/202...,고급,"•필수재료 : 느타리버섯(25g), 파프리카(빨간색, 노란색 각 10g), 청피망(...",1. 느타리버섯은 밑동을 자르고 가닥가닥 찢어 데쳐서 찬물에 헹궈 물기를 제거한다....,0,0
1108,떡꼬치구이,일반식,http://www.foodsafetykorea.go.kr/uploadimg/202...,고급,"•필수 재료 : 떡볶이떡(40g), 파(3.5g), 양파(7g), 식용유(3g)\n...",1. 떡볶이 떡은 끓는 물에 데쳐 건진다.\n2. 꼬치에 데친 떡을 꽂는다.\n3....,0,0


In [8]:
# 정제된 데이터를 csv파일로 저장
df.to_csv('brecipe_data.csv', encoding='utf-8', index=False)

In [ ]:
# csv파일을 json으로 변환
# json파일을 loader로 만들기
import json

with open("brecipe_data.json", "r", encoding="utf-8") as f:
  brecipes = json.load(f)

new_list = []
for brecipe in brecipes:
  new_data = {"model" : "recipe.baserecipe"}
  # pk는 autofield
  new_data["fields"] = brecipe
  new_list.append(new_data)

with open("baserecipe.json", "w", encoding="utf-8") as f:
  json.dump(new_list, f, ensure_ascii=False, indent=2)